IMPORTING LIBRARIES


In [ ]:
# General
import pickle
import itertools
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Pandas
import pandas as pd
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Numpy
import numpy as np

# Matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["font.family"] = "Sans"
plt.style.use('seaborn-poster')


# Seaborn
import seaborn as sns
sns.set()

# Sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score,GridSearchCV,train_test_split,cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import scale,StandardScaler,MinMaxScaler,Normalizer,RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier

# Sklearn Metrics and Evaluation
from sklearn.metrics import classification_report
from sklearn.metrics import  accuracy_score, f1_score,precision_score,confusion_matrix, recall_score, roc_auc_score

# LightGBM
from lightgbm import LGBMClassifier

#XGBoost
from xgboost import XGBClassifier

# CatBoost
from catboost import CatBoostClassifier

# Statistics Testing
from scipy import stats
from scipy.stats import f_oneway
from scipy.stats import ttest_ind
from scipy.stats import chi2_contingency

import scikitplot as skplt

Dataset Information, Feature Checking, and Analysis

In [ ]:
def dataInfo(X):
    print('='*10, 'DATA & FEATURE INFORMATION', '='*10)
    print(f'Data Rows: {X.shape[0]}')
    print(f'Data Columns: {X.shape[1]}')
    print(f'Duplicated Values: {X.duplicated().sum()}')
    missing_val = X.isnull().sum()
    missing_val_percentage = missing_val/len(X)*100
    data_type = X.dtypes
    unique_val = X.nunique()
    return pd.DataFrame({
        'Missing_val' : missing_val,
        'Missing_percentage' : missing_val_percentage,
        'Data_type' : data_type,
        'Unique_values' : unique_val
    }).sort_values('Missing_percentage',ascending=False)

Checking Outlier in Data and Returning It

In [ ]:
def dataOutlier(X, col):
    q1 = X[col].quantile(0.25) # Finding the first quartile
    q3 = X[col].quantile(0.75) # Finding the third quartile
    iqr_val = stats.iqr(X[col]) # Finding the interquartile range
    loYour = q1 - (1.5*iqr_val) # LoYour limit for detecting outliers
in the data


    upper = q3 + (1.5*iqr_val) # Upper limit for detecting outliers in the data

    outlier_list = X[col].apply(lambda x: 'outlier' if x < loYour or x > upper else 'not-outlier')
    print(f'outlier loYour limit: {loYour}\noutlier upper limit:{upper}')
    return outlier_list

Checking Outlier Using LOF method

In [ ]:
def lof_observation(df):
    # Define numeric data types
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    # Select numeric columns
    df_num_cols = df.select_dtypes(include=numerics)

    # Convert to float
    df_outlier = df_num_cols.astype("float64")

    # Initialize Local Outlier Factor
    clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)

    # Fit the model and get the outlier scores
    clf.fit_predict(df_outlier)
    df_scores = clf.negative_outlier_factor_

    # Create a DataFrame for outlier scores
    scores_df = pd.DataFrame(np.sort(df_scores))

    # Plot visualization of outliers
    scores_df.plot(stacked=True, xlim=[0, 20], color='r',
                   title='Visualization of outliers according to the LOF method', style='.-')

    # Determine the threshold value for outliers
    th_val = np.sort(df_scores)[2]

    # Identify outliers
    outliers = df_scores > th_val

    # Drop outliers from the original DataFrame
    df_cleaned = df.drop(df_outlier[~outliers].index)

    # Return the cleaned DataFrame shape
    return df_cleaned

Returning Unique Values in Data

In [ ]:
def dataUnique(X):
    for x in X.columns:
        print(f'========== {x} ==========')
        print(f'{X[x].unique()}\n') # Printing all the unique values in the columns

Data Visualization (Used Mostly to See The Data Distribution and
Outlier Existence)

In [ ]:
def hist_boxplot(df, x_col, y_col=None, hue_col=None):
    color = '#F84030'
    if y_col is None:
        plt.figure(figsize=(10,10))
        plt.subplot(211)
        sns.boxplot(data=df, x=x_col, color=color)

        plt.subplot(212)

        sns.histplot(data=df, x=x_col, color=color)
        plt.tight_layout()
        plt.show()

    elif hue_col is None:
        plt.figure(figsize=(10,10))
        plt.subplot(211)
        sns.boxplot(data=df, x=x_col, y=y_col, color=color)

        plt.subplot(212)
        sns.histplot(data=df, x=x_col, y=y_col, color=color)
        plt.tight_layout()
        plt.show()

    else:
        plt.figure(figsize=(10,10))
        plt.subplot(211)
        sns.boxplot(data=df, x=x_col, y=y_col, hue=hue_col, color=color)

        plt.subplot(212)
        sns.histplot(data=df, x=x_col, y=y_col, hue=hue_col,color=color)
        plt.tight_layout()
        plt.show()

Another Data Visualization (Used to See The Corelation of Certain
Variable, Mostly Between Predictor and Target)

In [ ]:
#catergoric vs categoric
#predictor,target
def plot_category_category(df, predictor, target):
    ax = ((df.groupby([predictor])[target].mean())/((df.groupby([predictor])[target].mean()).sum())*100).plot(kind='bar',figsize=(10,8), color ='#F84030')
    abs_values = round(((df.groupby([predictor])[target].mean())/((df.groupby([predictor])[target].mean()).sum())*100),1)ax.bar_label(container=ax.containers[0], labels=abs_values)

    plt.xticks(rotation=0);
    plt.ylabel('Churn Rate (%)',fontsize=12)
    plt.title(f'{predictor} vs Churn Rate (%)',fontsize=12)
    plt.xlabel(predictor,fontsize=12)
    plt.ylim(0,100)

    plt.show()

def plot_num_category(df, numeric, category):

    ax = (df.groupby([category])[numeric].median()).plot(kind='bar',figsize=(10,8), color = '#F84030')
    abs_values = round((df.groupby([category])[numeric].median()),1)
    ax.bar_label(container=ax.containers[0], labels=abs_values)

    plt.title(f'{numeric} vs {category}',fontsize=12)
    plt.ylabel(category,fontsize=12)
    plt.xlabel(numeric,fontsize=12)
    plt.xticks(rotation=0);
    plt.ylim(0,max(abs_values)+100)
    plt.show()

Statistical Analysis (Analysis of Variance and Chi-Square Test)

In [ ]:
def anova(df, predictor_col, target_col):
    group1 = df[df[target_col]==1][predictor_col]
    group2 = df[df[target_col]==0][predictor_col]

    alpha = 0.05
    ftest, p_value = f_oneway(group1, group2)

    if p_value < alpha:
        print(f'Conclusion: p-value {p_value} is less than alpha {alpha}, ftest = {ftest}')
        print('There is a significant difference in the target based on the predictor.')
        print()
    else:
        print(f'Conclusion: p-value {p_value} is greater than alpha {alpha}, ftest = {ftest}')
        print('There is no significant difference in the target based on the predictor.')
        print()
def chi_test(df, predictor_col, target_col):
    obs = pd.crosstab(df[predictor_col], df[target_col])
    chi2, p, dof, expected = chi2_contingency(obs)
    alpha = 0.05

    if p < alpha:
        print(f'===== {predictor_col} =====')
        print('Number Expected:')
        print(expected)
        print()
        print(f'Chi-square Statistic: {chi2}, p-value: {p} less than{alpha}')
        print('There is a significant difference in the target based on the predictor.')
        print()
    else:
        print(f'===== {predictor_col} =====')
        print('Number Expected:')

        print(expected)
        print()
        print(f'Chi-square Statistic: {chi2}, p-value: {p} greater than {alpha}')
        print('There is no significant difference in the target based on the predictor.')
        print()

##Data Preparation

Import data

In [ ]:
data = pd.read_csv('/content/Churn_Modelling.csv')
TARGET = "Exited"
data_new = data.copy()
data_new['Tenure'] =  data_new.Tenure.astype(float)
data_new['NumOfProducts'] =  data_new.NumOfProducts.astype(float)

Data Descriptive Statistics

In [ ]:
numerical = ['CreditScore',
             'Age',
             'Tenure',
             'Balance',
             'NumOfProducts',
             'EstimatedSalary']
categorical =['RowNumber',
              'CustomerId',
              'Surname',
              'Geography',
              'Gender',
              'HasCrCard',
              'IsActiveMember',
              'Exited']
# Select rows where is 1 (Exited)
df_zero = data_new.query('Exited == 1')

# Count occurrences of each value
counts = df_zero['Exited'].value_counts()

plot_exited = pd.DataFrame({'Yes' : counts,'No' : len(data_new) -
counts})

# Plot as pie chart
plot_exited.T.plot(kind = 'pie',subplots=True, legend =False, title =
'Exited', figsize = (6,6),colors=['#F84030','#F2F2F2'],
        shadow=True,
        autopct='%1.1f%%',
        textprops={'fontsize': 16})

##Exploratory Data Analysis

In [ ]:
lof_observation(data_new).head()

Profile Analysis of The Customer

1. CreditScore vs Exited

In [ ]:
hist_boxplot(data_new,'CreditScore')

2. Age vs Exited

In [ ]:
hist_boxplot(data_new,'Age')

3. Tenure vs Exited

In [ ]:
hist_boxplot(data_new,'Tenure')

4. NumOfProducts vs Exited

In [ ]:
hist_boxplot(data_new,'NumOfProducts')

5. EstimatedSalary vs Exited

In [ ]:
hist_boxplot(data_new,'EstimatedSalary')

6. Geography vs Exited

In [ ]:
plot_category_category(data_new,'Geography',TARGET)

7. HasCrCard vs Exited

In [ ]:
plot_category_category(data_new,'HasCrCard',TARGET)

8. IsActiveMember vs Exited

In [ ]:
plot_category_category(data_new,'IsActiveMember',TARGET)

9.Gender vs Exited

In [ ]:
plot_category_category(data_new,'Gender',TARGET)

Outlier Removal and Data Cleaning

In [ ]:
def clear_outliers(df):
    # Suppress age and credit score
    for col in ["Age", "CreditScore"]:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        loYour = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        print(f"When {col} is printed below loYour score: {loYour} and upper score: {upper}")
        if col == "Age":
            df_outlier = df[col][(df[col] > upper)]
            df[col][df_outlier.index] = upper
        else:
            df_outlier = df[col][(df[col] < loYour)]
            df[col][df_outlier.index] = loYour
    return df

def outlier_process(df):
    # Process outliers
    df_outlier = lof_observation(df=df)
    df_outlier = clear_outliers(df=df_outlier)
    return df_outlier

##Feature Engineering

Credit Score

In [ ]:
def credit_score_table(row):
    credit_score = row.CreditScore
    if credit_score < 300:
        return "Deep"
    elif credit_score < 500:
        return "Very_Poor"
    elif credit_score < 601:
        return "Poor"
    elif credit_score < 661:
        return "Fair"
    elif credit_score < 781:
        return "Good"
    elif credit_score < 851:
        return "Excellent"
    else:
        return "Top"

Age

In [ ]:
def age_categorization(age):
    if age < 0:
        return "Invalid"
    elif age < 13:
        return "Child"
    elif age < 18:
        return "Teenager"
    elif age < 30:
        return "Young Adult"
    elif age < 60:
        return "Adult"
    else:
        return "Senior"

Age and Tenure

In [ ]:
def age_tenure(row):
    age = row.Age
    tenure = row.Tenure
    return age*tenure

Product and Tenure

In [ ]:
def product_utilization_rate_by_year(row):
    number_of_products = row.NumOfProducts
    tenure = row.Tenure

    if tenure == 0:
        return number_of_products
    else:
        rate = number_of_products / tenure
        return rate

Product and Salary

In [ ]:
def product_utilization_rate_by_estimated_salary(row):
    number_of_products = row.NumOfProducts
    estimated_salary = row.EstimatedSalary

    if estimated_salary == 0:
        return "Undefined"
    else:
        rate = number_of_products / estimated_salary
        return rate

Geography and Salary

In [ ]:
def countries_monthly_average_salaries(row):
    # Monthly average salary taken from
https://www.worlddata.info/average-income.php
    avg_salaries = {'France': 3680, 'Germany': 4305, 'Spain': 2474}

    salary = row.EstimatedSalary / 12
    country = row.Geography

    if country in avg_salaries:
        return salary / avg_salaries[country]
    else:
        return None

Balance and Salary

In [ ]:
def balance_salary(row):
    balance = row.Balance
    salary = row.EstimatedSalary
    return balance/salary

Credit Score Squared

In [ ]:
def credit_score_squared(row):
    return (row.CreditScore)**2

Mean Financial Condition (Financial Health)

In [ ]:
def mean_financials(row):
    credit_score = row.CreditScore
    balance = row.Balance
    estimated_salary = row.EstimatedSalary

    mean_financials = (credit_score + balance + estimated_salary) / 3
    return mean_financials

All Feature Engineering

In [ ]:
def feature_engineering(df, is_show_graph=False):
    df_fe = df.copy()

    # balance_salary_rate
    df_fe['balance_salary_rate'] = df_fe.apply(lambda x:balance_salary(x), axis=1)

    # product_utilization_rate_by_year
    df_fe = df_fe.assign(product_utilization_rate_by_year=df_fe.apply(lambda x:product_utilization_rate_by_year(x), axis=1))

    # product_utilization_rate_by_estimated_salary
    df_fe = df_fe.assign(product_utilization_rate_by_estimated_salary=df_fe.apply( lambda x: product_utilization_rate_by_estimated_salary(x), axis=1))

    # tenure_rate_by_age
    df_fe['tenure_rate_by_age'] = df_fe.Tenure / (df_fe.Age - 17)

    # credit_score_rate_by_age
    df_fe['credit_score_rate_by_age'] = df_fe.CreditScore / (df_fe.Age - 17)

    # product_utilization_rate_by_salary
    df_fe['product_utilization_rate_by_salary'] = df_fe.Tenure /(df_fe.EstimatedSalary)

    # credit_score_rate_by_salary
    df_fe['credit_score_rate_by_salary'] = df_fe.CreditScore /(df_fe.EstimatedSalary)

    # mean_financials
    df_fe['mean_financials'] = df_fe.apply(lambda x: mean_financials(x), axis=1)

    # credit_score_squared
    df_fe['credit_score_squared'] = df_fe.apply(lambda x: credit_score_squared(x), axis=1)

    # age_tenure
    df_fe['age_tenure'] = df_fe.apply(lambda x: age_tenure(x), axis=1)

    # age_categorization
    df_fe['age_categorization'] = df_fe.Age.apply(lambda x: age_categorization(x))

    # credit_score_table
    df_fe = df_fe.assign(credit_score_table=df_fe.apply(lambda x:credit_score_table(x), axis=1))

    # countries_monthly_average_salaries
    df_fe = df_fe.assign(countries_monthly_average_salaries=df_fe.apply(lambda x: countries_monthly_average_salaries(x), axis=1))

    if is_show_graph:
        fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20, 12))
        fig.tight_layout()

        sns.boxplot(y='balance_salary_rate', x=TARGET, hue=TARGET, data=df_fe, ax=axes[0][0])
        sns.boxplot(y='product_utilization_rate_by_year', x=TARGET, hue=TARGET, data=df_fe, ax=axes[0][1])
        plt.ylim(-1, 5)

    return df_fe

Data Encoding

In [ ]:
def data_encoding(df):
    df_model = df.copy()

    # Categorical columns
    non_encoding_columns = ["Geography", "HasCrCard", "IsActiveMember", "Gender", "NumOfProducts", "Tenure", "credit_score_table","age_categorization"]
    df_non_encoding = df_model[non_encoding_columns]
    df_model = df_model.drop(non_encoding_columns, axis=1)

    df_encoding = df_non_encoding.copy()

    from sklearn.preprocessing import LabelEncoder
    encoder = LabelEncoder()
    df_encoding["gender_category"] = encoder.fit_transform(df_non_encoding.Gender)
    df_encoding["country_category"] = encoder.fit_transform(df_non_encoding.Geography)
    df_encoding["credit_score_category"] = encoder.fit_transform(df_non_encoding.credit_score_table)
    df_encoding["age_category"] = encoder.fit_transform(df_non_encoding.age_categorization)

    df_encoding.reset_index(drop=True, inplace=True)
    df_model.reset_index(drop=True, inplace=True)
    df_model = pd.concat([df_model, df_encoding], axis=1)

    # Drop unnecessary columns
    df_model = df_model.drop(["Geography", "Gender", "CustomerId", "Surname", "credit_score_table","age_categorization"], axis=1)

    # Encode HasCrCard and IsActiveMember as -1 for 0 values
    df_model.loc[df_model.HasCrCard == 0, 'HasCrCard'] = -1
    df_model.loc[df_model.IsActiveMember == 0, 'IsActiveMember'] = -1

    return df_model

data_encoded = data_encoding(data_feature)
data_encoded.head(20)

Correlation Analysis

In [ ]:
correlation = data_encoded.corr().abs()
plt.figure(figsize=(20,20))
sns.heatmap(correlation, annot=True)
plt.show()

Model Utilization

In [ ]:
def model_prepare(df_model):
    y = df_model[TARGET]
    X = df_model.drop(TARGET, axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=32)

    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    return X_train, X_test, y_train, y_test
# model_prepare test, train split 0.3
X_train, X_test, y_train, y_test = model_prepare(df_model = data_encoded)

Logistic Regression

In [ ]:
logr_model = LogisticRegression()
logr_model.fit(X_train, y_train)
y_pred = logr_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(classification_report(y_test, y_pred, digits=4))
print(f"Accuracy score of Logistic Regression: {accuracy:.4f}")

Model with Training

In [ ]:
def data_training(X_train, X_test, y_train, y_test):
    models = [
        ('LOGR', LogisticRegression()),
        ('KNN', KNeighborsClassifier()),
        ('CART', DecisionTreeClassifier()),
        ('RF', RandomForestClassifier()),
        ('GBM', GradientBoostingClassifier()),
        ('XGBoost', XGBClassifier()),
        ('LightGBM', LGBMClassifier()),
        ('CatBoost', CatBoostClassifier(verbose=0))
        ]


    df_result = pd.DataFrame(columns=["model", "accuracy_score", "scale_method", "0_precision", "0_recall", "1_precision", "1_recall"])
    for name, model in models:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        score = accuracy_score(y_test, y_pred)
        class_report = classification_report(y_test, y_pred, digits=2, output_dict=True)
        zero_report = class_report['0']
        one_report = class_report['1']
        df_result = pd.concat([df_result, pd.DataFrame({
            'model': [name],
            'accuracy_score': [score],
            'scale_method': ['NA'],
            '0_precision': [zero_report['precision']],
            '0_recall': [zero_report['recall']],
            '1_precision': [one_report['precision']],
            '1_recall': [one_report['recall']]
        })], ignore_index=True)

    return
    df_result.sort_values("accuracy_score", ascending=False)
    training_result = data_training(X_train, X_test, y_train, y_test)
    training_result

##Hyperparameter Tuning

XGBoost

In [ ]:
xgb_model = XGBClassifier(objective='reg:logistic',
eval_metric="logloss")

param_grid = {
    'learning_rate': [0.3, 0.31, 0.32],
    'max_depth': [5,6,7],
    'n_estimators': [30,40, 50]
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid,
cv=5)
grid_search.fit(X_train, y_train)

print("Best parameters: ", grid_search.best_params_)


xgb_model=XGBClassifier(silent=0, learning_rate=0.31,
max_delta_step=5,
                            objective='reg:logistic',n_estimators=30,
77

                            max_depth=3, eval_metric="logloss",
gamma=3,base_score=0.5)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
print(classification_report(y_test,y_pred,digits=2))
print("Accuracy score of Tuned XGBoost Regression: ",
accuracy_score(y_test, y_pred))

Random Forest

In [ ]:
param_grid = {'max_depth': [6, 7, 8], 'max_features':
[7,8,9],'n_estimators' : [50,100], 'min_samples_split': [5, 6, 7]}
randFor_grid = GridSearchCV(RandomForestClassifier(), param_grid, cv =
3, refit = True, verbose = 0)
randFor_grid.fit(X_train,y_train)
y_pred = randFor_grid.predict(X_test)
print(classification_report(y_test,y_pred,digits=2))
print("Accuracy score of tuned Random Forest model: ",
accuracy_score(y_test, y_pred))
rnd_model = RandomForestClassifier(max_depth=10, max_features=14,
min_samples_split=11,n_estimators=75)
rnd_model.fit(X_train, y_train)
y_pred = rnd_model.predict(X_test)
print(classification_report(y_test,y_pred,digits=2))
print("Accuracy score of tuned Random Forest model: ",
accuracy_score(y_test, y_pred))

LightBGM

In [ ]:
lgbm_model = LGBMClassifier(silent = 0, learning_rate = 0.08,
max_delta_step = 1, n_estimators = 45, boosting_type = 'gbdt',
                            max_depth = 5)
lgbm_model.fit(X_train, y_train)
y_pred = lgbm_model.predict(X_test)
print(classification_report(y_test, y_pred, digits=2))
print("Accuracy score of tuned LightGBM model: ", accuracy_score(y_test,
y_pred))

Parameter and More Function

In [ ]:
# Function to display the best model score and parameters
def display_best_model(model):
    print(f"Best score: {model.best_score_}")
    print(f"Best parameters: {model.best_params_}")
    print(f"Best estimator: {model.best_estimator_}")

# Function to calculate AUC score and ROC curve
def calculate_auc_scores(y_actual, y_pred_prob):
    auc_score = roc_auc_score(y_actual, y_pred_prob)
    fpr, tpr, _ = roc_curve(y_actual, y_pred_prob)
    return auc_score, fpr, tpr
78


# Function to plot confusion matrix
def plot_confusion_matrix(cm, classes, normalize=False,
title='Confusion matrix', cmap=plt.cm.Blues):
    plt.rcParams.update({'font.size': 16})

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, color="blue")
    plt.yticks(tick_marks, classes, color="blue")

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="red" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Function to handle infinite values in a DataFrame
def handle_infinite_values(df):
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

Confussion Matrix

In [ ]:
cfm = confusion_matrix(y_test, y_pred=y_pred)
plot_confusion_matrix(cfm, classes=['Non Churn','Churn'],
                      title='Churn Confusion matrix')
tn, fp, fn, tp = cfm.ravel()
print("True Negatives: ",tn)
print("False Positives: ",fp)
print("False Negatives: ",fn)
print("True Positives: ",tp)

ROC Curve

In [ ]:
y_pred_proba = lgbm_model.predict_proba(X_test)
skplt.metrics.plot_roc_curve(y_test, y_pred_proba, figsize=(8,8))

plt.show()

Feature Importance

In [ ]:
feature_index = data_encoded.loc[:, data_encoded.columns != TARGET]

feature_importance = pd.Series(xgb_model.feature_importances_,
                               index=feature_index.columns).sort_value
 s(ascending=False)
sns.barplot(x = feature_importance, y = feature_importance.index,
color='r', saturation=1)
plt.xlabel('Variable Severity Scores')
plt.ylabel('Variables')
plt.title('Variable Severity Levels')
plt.show()

Pipeline

In [ ]:
from sklearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = model_prepare(df_model =
data_encoded)

pipe = Pipeline([('xgboost', xgb_model)])

pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)

print(classification_report(y_test,y_pred,digits=2))
print("Accuracy score of Tuned XGBoost Regression: ",
accuracy_score(y_test, y_pred))

with open('pipeline.pickle','wb') as f:
    pickle.dump(pipe, f)

with open('pipeline.pickle','rb') as f:
    loaded_pipe = pickle.load(f)

y_pred = loaded_pipe.predict(X_test)

print(classification_report(y_test,y_pred,digits=2))
80

print("Accuracy score of Tuned XGBoost Regression: ",
accuracy_score(y_test, y_pred))

def prediction(model, df):
    data_feature = feature_engineering(df)
    data_encoded = data_encoding(data_feature)

    prediction = model.predict(data_encoded)

    df['ExitedPred'] = prediction.tolist()

    return prediction
with open('churn_model.pkl','wb') as f:
    pickle.dump(xgb_model, f)

with open('churn_model.pkl','rb') as f:
    loaded_pipe = pickle.load(f)

y_pred = loaded_pipe.predict(X_test)

print(classification_report(y_test,y_pred,digits=2))
print("Accuracy score of Tuned XGBoost Regression: ",
accuracy_score(y_test, y_pred))